# Preprocess FaceID Dataset

In [ ]:
import os, sys, shutil, requests, zipfile, io, re
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm as pbar
from PIL import Image 
sys.path.insert(0, '/home/huy/Projects/FaceID')

In [ ]:
data_dir = '/data/huy/faceid/'

## Download dataset

Dataset link http://www.vap.aau.dk/rgb-d-face-database/

In [ ]:
train_url = 'https://rutgers.box.com/shared/static/vfaxy3y1otrvn5dbvbxdg3unmvyun7yg.zip'
val_url = 'https://rutgers.box.com/shared/static/smgbvw9v5qemc196072z51x7kwfaxn4b.zip'

In [ ]:
def download(url):
    # Streaming, so we can iterate over the response.
    r = requests.get(url, stream=True)

    # Total size in Mebibyte
    total_size = int(r.headers.get('content-length', 0))
    block_size = 2**20 # Mebibyte
    t = pbar(total=total_size, unit='MiB', unit_scale=True)

    header = r.headers['content-disposition']
    file_name = re.findall("filename\*=UTF-8\'\'(.+)", header)[0]

    with open(file_name, 'wb') as f:
        for data in r.iter_content(block_size):
            t.update(len(data))
            f.write(data)
    t.close()

    if total_size != 0 and t.n != total_size:
        raise Exception('Error, something went wrong')
        
    print('Download successful. Unzipping ' + file_name)
    file_path = os.path.join(os.getcwd(), file_name)
    
    extract_dir = os.getcwd()
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
        print('Unzip file successful!')

In [ ]:
download(val_url)

## Process data

In [ ]:
def crop_center(img, cropx, cropy):
    y,x = img.shape[0], img.shape[1]
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [ ]:
def process_data(raw_dir, processed_dir):
    """
    Stack RGB and D images to a numpy array of shape 400 x 400 x 4
    Save images in format of personID_poseID
    """    
    all_people = [x for x in os.listdir(raw_dir) if '2012' in x]
    for person_id, a_person in enumerate(pbar(all_people)):
        
        all_poses = [x for x in os.listdir(os.path.join(raw_dir, a_person)) if '.bmp' in x]
        for pose_id, a_pose in enumerate(all_poses):
            photo_id = a_pose[:-5] # drop 'c.bmp'

            photo_rgb = photo_id + 'c.bmp'
            photo_rgb = Image.open(os.path.join(raw_dir, a_person, photo_rgb))
            photo_rgb = np.array(photo_rgb.convert("RGB").resize((640, 480)))

            photo_depth = photo_id + 'd.dat'
            photo_depth = np.loadtxt(os.path.join(raw_dir, a_person, photo_depth))
            photo_depth = np.where((photo_depth > 400) & (photo_depth < 3000), photo_depth, 0) # Valid range is from 400 to 3000
            photo_depth = (photo_depth - 400 / (3000 - 400)) * 255.0
            photo_depth = np.expand_dims(photo_depth, -1)
            
            # Save to disk
            rgbd = np.concatenate((photo_rgb, photo_depth), axis=2)
            rgbd = np.uint8(rgbd)
            rgbd = crop_center(rgbd, 400, 400)
            name = 'person' + str(person_id) + '_pose' + str(pose_id) + '.npy'
            np.save(os.path.join(processed_dir, name), rgbd)

In [ ]:
os.mkdir('train')
process_data('train_raw', 'train')

os.mkdir('val')
process_data('val_raw', 'val')

In [ ]:
# Move data to desire place
os.mkdir(data_dir)
shutil.move('train_raw', data_dir)
shutil.move('val_raw', data_dir)
shutil.move('train', data_dir)
shutil.move('val', data_dir)

# Display a test RGBD image

In [ ]:
test_img_path = os.path.join(data_dir, 'train/person0_pose0.npy')
test_img = np.load(test_img_path)

In [ ]:
plt.imshow(test_img[:,:,:3]/255.0)
plt.show()

In [ ]:
plt.imshow(test_img[:,:,3]/255.0, cmap='jet')
plt.show()